<a href="https://colab.research.google.com/github/manashpratim/Bosch-Summer-Internship/blob/master/VideoV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q '/content/drive/My Drive/mosi_data/mosi.zip'

In [ ]:
def get_file_names(mypath,savefile):
  from os import listdir
  from os.path import isfile, join
  onlyfiles = [f[:f.find('.')] for f in listdir(mypath) if isfile(join(mypath, f))]
  with open(savefile, 'w') as f:
    for item in onlyfiles:
        f.write(item)
        f.write('\n')
  return onlyfiles

In [ ]:
mypath = '/content/Raw/Transcript/Segmented'
files = get_file_names(mypath,'textfile.txt')

In [ ]:
import pickle
with open('/content/drive/My Drive/mosi_data/video_frames_joined.pickle', 'rb') as handle:
    nd = pickle.load(handle)

with open('/content/drive/My Drive/mosi_data/labels_joined.pickle', 'rb') as handle:
    labels = pickle.load(handle)

In [ ]:
# This block pads the video features so that each segments have same length. I use zero padding
video_data = []
maximum = float('-inf')

for key in files:
  for l in nd[key]:
        video_data.append(l)
import tensorflow as tf
video_data=tf.keras.preprocessing.sequence.pad_sequences(video_data, maxlen=40, dtype='float32', padding='post', truncating='post',value=0.0)

In [ ]:
# Join all the labels into a numpy array
import numpy as np
y = []
for key in files:
    y+=labels[key]
y = np.array(y)

#For regression, do not execute any of the lines below

y[y>0]=1        #Convert labels to binary
y[y<0]=0

y=y.astype(int)   

#ref = {-3:0,-2:1,-1:2,0:3,1:4,2:5,3:6}         #Uncomment the following three lines for 7 class classification
#for i,num in enumerate(y):
#  y[i] = ref[num]

In [ ]:
def split_data(data,labels,split_size=0.8):
  import numpy as np  
  train_length =int(len(labels)*split_size)
  test_length =int(len(labels)-train_length)
  idx = np.random.permutation(labels.shape[0])
  data = data[idx]
  labels = labels[idx]
  data_train = data[:train_length]
  data_val = data[train_length:]
  labels_train = labels[:train_length]
  labels_val = labels[train_length:]
  
  return data_train,data_val,labels_train,labels_val

In [ ]:
video_train,video_val,labels_train,labels_val = split_data(video_data,y,split_size=0.8)
del video_data

In [ ]:
# video model
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model


inp = Input(shape=(40, 64, 64, 3))
x   = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), strides=(1,2,2),activation="relu", padding="same")(inp)
x   = tf.keras.layers.Conv3D(filters=64, kernel_size=(3, 3, 3), strides=(1,2,2),activation="relu", padding="same")(x)
x   = tf.keras.layers.BatchNormalization()(x)
x   = tf.keras.layers.Dropout(0.4)(x)
x   = tf.keras.layers.Reshape((40, 16*16*64))(x)


x = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128,return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128,return_sequences=True))(x)
x   = tf.keras.layers.BatchNormalization()(x)
x   = tf.keras.layers.Dropout(0.4)(x)
x  = tf.keras.layers.GlobalAveragePooling1D()(x)
x   = tf.keras.layers.BatchNormalization()(x)
x   = tf.keras.layers.Dropout(0.4)(x)
x   = tf.keras.layers.Dense(64, activation='relu')(x)
x   = tf.keras.layers.Dropout(0.5)(x)
out   = tf.keras.layers.Dense(1, activation='sigmoid')(x)
#out   = tf.keras.layers.Dense(7, activation='softmax')(x)
#out   = tf.keras.layers.Dense(1)(x)

model = Model(inp,out)

In [ ]:
reduce =tf. keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, mode='auto')
#early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, mode='auto')

#Uncomment one of the next three lines at a time               
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])         #Binary classification
#model.compile(loss='sparse_categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])            #7 class classifiaction
#model.compile(loss="mean_absolute_error",optimizer='rmsprop')             #Regression
history=model.fit(video_train,labels_train, batch_size=32, epochs=80, shuffle=True,validation_data=(video_val,labels_val),callbacks=[reduce],use_multiprocessing=True)